In [11]:
import requests
import pandas as pd
import time

url = "http://135.125.84.175" # cette url peut éventuellement changer !!

# le dataframe des publications
df = pd.read_csv('/Users/eric/Downloads/publis_lorraine_completes.csv')

# s'il y a un scientific field restant, on le supprime (il va etre recalculé )
if 'scientific_field' in df:
    del df['scientific_field']

# le classifier n'a besoin que du titre et journal_name (pour le moment)
# le doi servira à recroiser les résultats in fine

publis_for_classif = df[['doi', 'title', 'journal_name']].to_dict(orient='records')
nb_publis = len(publis_for_classif)
print(f"sending {nb_publis} to classifier")
task_id = requests.post(f"{url}/classify", json={"type": "bso", "publications": publis_for_classif}).json()['data']['task_id']
print("classification", end=':')
for i in range(1000):
    print(i, end='...')
    r = requests.get(f"{url}/tasks/{task_id}").json()
    if r.get('data', {}).get('task_status') == "started":
        time.sleep(10)
    elif r.get('data', {}).get('task_status') == "finished":
        break
df_classification = pd.DataFrame(r['data']['task_result'])
df_classification['scientific_field'] = df_classification['bso_classification']
nb_results = len(df_classification)
print()
print(f"{nb_results} results back from classif")

# on recroise le dataframe original avec les résultats
df_final = df.merge(df_classification[['doi', 'scientific_field']], on='doi', how='left')

sending 21457 to classifier
classification:0...1...2...3...
21457 results back from classif


In [12]:
df_final

,doi,oa_type,is_oa,title,published_date,published_year,genre,journal_is_in_doaj,journal_issns,journal_name,publisher,repositories,scientific_field
0,10.1016/j.chemgeo.2016.10.031,closed,False,Zinc isotope fractionation during adsorption t...,2016-12-01,2016.0,journal-article,False,0009-2541,Chemical Geology,Elsevier BV,NaN,"Earth, Ecology, \nEnergy and applied biology"
1,10.1371/journal.pone.0168349,publisher;repository,True,Prediction of Left Ventricular Remodeling afte...,2016-12-30,2016.0,journal-article,True,1932-6203,PLOS ONE,Public Library of Science (PLoS),europepmc.org;pdfs.semanticscholar.org;www.ncb...,Medical research
2,10.1016/j.jpowsour.2016.10.037,closed,False,A critical review on gas diffusion micro and m...,2016-12-01,2016.0,journal-article,False,0378-7753,Journal of Power Sources,Elsevier BV,NaN,Engineering
3,10.1016/j.jpowsour.2016.10.035,closed,False,Decohesion and rupture mechanisms of a multila...,2016-12-01,2016.0,journal-article,False,0378-7753,Journal of Power Sources,Elsevier BV,NaN,Engineering
4,10.1021/acs.jpcc.6b09974,publisher,True,Local Structure-Driven Localized Surface Plasm...,2016-12-19,2016.0,journal-article,False,"1932-7447,1932-7455",The Journal of Physical Chemistry C,American Chemical Society (ACS),NaN,"Physical sciences, Astronomy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21452,10.1016/j.postharvbio.2020.111350,publisher,True,"SlMAPK3, a key mitogen-activated protein kinas...",2021-01-01,2021.0,journal-article,False,0925-5214,Postharvest Biology and Technology,Elsevier BV,NaN,Biology (fond.)
21453,10.1101/2020.06.02.128975,repository,True,Paleoclimate and current climate collectively ...,2020-06-03,2020.0,posted-content,False,NaN,NaN,Cold Spring Harbor Laboratory,pure.mpg.de;www.biorxiv.org,Medical research
21454,10.1007/978-3-030-39165-2_21,closed,False,Product Innovation Determinants and Export Per...,2020-01-01,2020.0,book-chapter,False,"2363-6165,2363-6173",Developments in Marketing Science: Proceedings...,Springer International Publishing,NaN,Biology (fond.)
21455,10.1016/j.foreco.2020.118672,closed,False,Growth dynamics of fast-growing tree species i...,2021-01-01,2021.0,journal-article,False,0378-1127,Forest Ecology and Management,Elsevier BV,NaN,Biology (fond.)
